In [2]:
import json
from datasets import Dataset

In [3]:
file_path = "codenet-c-traces-line.jsonl"
data = []
with open(file_path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        try:
            data.append(json.loads(line))
        except json.JSONDecodeError as e:
            print(f"Errore JSON alla riga {i+1}: {e}")
            continue

print(f"✅ Caricati {len(data)} esempi validi")


✅ Caricati 188026 esempi validi


In [18]:
processed_data = []

for idx, example in enumerate(data):
    code_lines = example["src_lines"]
    input_str = example["input"]
    execution_counts = example["count_in_trace"]

    # Mostra un esempio prima e dopo
    if idx < 2:
        print("🔹 Codice originale:")
        for line in code_lines:
            print(line)
        print("Input:", input_str)
        print("Line Execution:", execution_counts)

    formatted_code = '\n'.join(code_lines)
    num_lines = len(execution_counts)
    model_input = f"code: {formatted_code} <SEP> input: {input_str} <SEP> lines: {num_lines}"
    model_output = f"execution: {execution_counts}"

    processed_data.append({
        "input": model_input,
        "target": model_output
    })

    if idx < 2:
        print("\n✅ Dopo il preprocessing:")
        print("Input modello:", model_input)
        print("Target modello:", model_output)


✅ Dopo il preprocessing:
Input modello: code: #include <stdio.h>
int main(void) {
int n;
int h[20];
int i;
int x=0;
int max=0;
scanf("%d\n", &n);
for(i=0;i<n;i++){
scanf("%d ",&h[i]);
}
for(i=0;i<=n;i++){
if(h[max]<=h[i+1]){
x++;
max=i+1;
}
}
printf("%d\n",x);
return 0;
} <SEP> input: 4
6 5 6 8 <SEP> lines: 20
Target modello: execution: [0, 1, 0, 0, 0, 1, 1, 1, 5, 4, 0, 6, 5, 3, 3, 0, 0, 1, 1, 1]

✅ Dopo il preprocessing:
Input modello: code: #include <stdio.h>
int main(void) {
int n;
int h[20];
int i;
int x=0;
int max=0;
scanf("%d\n", &n);
for(i=0;i<n;i++){
scanf("%d ",&h[i]);
}
for(i=0;i<=n;i++){
if(h[max]<=h[i+1]){
x++;
max=i+1;
}
}
printf("%d\n",x);
return 0;
} <SEP> input: 4
6 5 6 8 <SEP> lines: 20
Target modello: execution: [0, 1, 0, 0, 0, 1, 1, 1, 5, 4, 0, 6, 5, 3, 3, 0, 0, 1, 1, 1]


In [5]:
with open("train.json", "w", encoding="utf-8") as f:
    json.dump(processed_data, f, indent=2)

print("File `train.json` salvato correttamente.")

File `train.json` salvato correttamente.


In [ ]:
!python finetune_codet5p.py \
  --model-name Salesforce/codet5p-220m \
  --cache-data train.json \
  --save-dir codet5p_output \
  --batch-size 4 \
  --epochs 1 \
  --data-num 1000

2025-05-31 05:30:24.143772: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748669424.180483     657 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748669424.194735     657 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-31 05:30:24.231901: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
tokenizer_config.json: 100% 1.48k/1.48k [00:00<00:00, 6.26MB/s]
vocab.json: 100% 703k/703k [00:00<00:00, 12.7MB/s]
merges.txt

In [ ]:
!zip -r codet5p_output.zip codet5p_output

  adding: codet5p_output/ (stored 0%)
  adding: codet5p_output/checkpoint-200/ (stored 0%)
  adding: codet5p_output/checkpoint-200/tokenizer.json (deflated 82%)
  adding: codet5p_output/checkpoint-200/tokenizer_config.json (deflated 94%)
  adding: codet5p_output/checkpoint-200/model.safetensors (deflated 7%)
  adding: codet5p_output/checkpoint-200/special_tokens_map.json (deflated 97%)
  adding: codet5p_output/checkpoint-200/generation_config.json (deflated 33%)
  adding: codet5p_output/checkpoint-200/scheduler.pt (deflated 56%)
  adding: codet5p_output/checkpoint-200/rng_state.pth (deflated 24%)
  adding: codet5p_output/checkpoint-200/config.json (deflated 49%)
  adding: codet5p_output/checkpoint-200/training_args.bin (deflated 52%)
  adding: codet5p_output/checkpoint-200/merges.txt (deflated 54%)
  adding: codet5p_output/checkpoint-200/trainer_state.json (deflated 75%)
  adding: codet5p_output/checkpoint-200/vocab.json (deflated 59%)
  adding: codet5p_output/checkpoint-200/optimizer.

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r codet5p_output /content/drive/MyDrive/

In [6]:
# Creazione dataset per fare inferenza
with open("train.json", "r", encoding="utf-8") as f:
    all_data = json.load(f)

dataset = Dataset.from_list(all_data)

split = dataset.train_test_split(test_size=0.2, seed=42)
test_data = split["test"]

test_list = test_data.to_list()

# Salva il test set su file
with open("test.json", "w", encoding="utf-8") as f:
    json.dump(test_list, f, indent=2)

print(f"Test set salvato con {len(test_list)} esempi in test.json")

Test set salvato con 37606 esempi in test.json


In [15]:
!cp -r /content/drive/MyDrive/codet5p_output /content/

In [16]:
!python evaluate_codet5p.py \
  --model-path codet5p_output/final_checkpoint \
  --test-file test.json \
  --save-preds

2025-06-02 15:05:56.575053: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748876756.603657    6327 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748876756.609985    6327 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-02 15:05:56.634845: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
🔍 Inizio inferenza su 200 esempi...
`generation_config` default values have been modified to match model-specific defaults: {